In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot

import keras
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

import gc

In [ ]:
def cnn_data_sequence_pre_train(data, target, seque_len,previo_aniadido=False):
    print(data.shape)
    print(target.shape)
    new_data = []
    new_target = []
    if(previo_aniadido == False):
        for index in range(data.shape[0] - seque_len + 1):
            new_data.append(data[index: index + seque_len])
            new_target.append(target[index + seque_len-1])
            print(index,':',index + seque_len - 1,'-->',index + seque_len-1)
    else:
        for index in range(data.shape[0] - seque_len + 1):
            new_data.append(data[index: index + seque_len])
            new_target.append(target.iloc[index])
            print(index,':',index + seque_len - 1,'-->',index)

    new_data = np.array(new_data)
    new_target = np.array(new_target)

    new_data = new_data.reshape(new_data.shape[0], new_data.shape[1], new_data.shape[2], 1)

    return new_data, new_target


def mostrarEvolucion(hist):
    loss = hist.history['loss']
    val_loss = hist.history['val_loss']
    pyplot.plot(loss)
    pyplot.plot(val_loss)
    pyplot.legend(['Training loss', 'Validation loss'])
    pyplot.show()

    acc = hist.history['accuracy']
    val_acc = hist.history['val_accuracy']
    pyplot.plot(acc)
    pyplot.plot(val_acc)
    pyplot.legend(['Training accuracy','Validation accuracy'])
    pyplot.show()

### Cargamos el conjunto de datos

In [ ]:
# Cargamos los datos originales
df = pd.read_csv('../spyconalfas.csv')
# Separamos en X - (conjunto de entrenamiento) y Y - (etiquetas de los movimientos)
Y = df['next movement']
X = df
X.drop(['next movement','alpha53'],axis=1,inplace=True)
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,shuffle=False)

fecha_inicio_train = X_train['Date'].iloc[0]
fecha_fin_train = X_train['Date'].iloc[-1]
fecha_inicio_test = X_test['Date'].iloc[0]
fecha_fin_test = X_test['Date'].iloc[-1]

print('Intervalo de entrenamiento:',fecha_inicio_train,'-',fecha_fin_train)
print('Intervalo de prueba:',fecha_inicio_test,'-',fecha_fin_test)

X_train_open = X_train['Open']
X_test_open = X_test['Open']
X_train_close = X_train['Close']
X_test_close = X_test['Close']

X_train_ret = X_train['next returns']
X_test_ret = X_test['next returns']
X_train.drop(['next returns', 'Date', 'Open'], axis=1, inplace=True)
X_test.drop(['next returns', 'Date', 'Open'], axis=1, inplace=True)

cols_to_norm = ['Close','alpha54','alpha84']
scaler  = StandardScaler()

X_train[cols_to_norm] = scaler.fit_transform(X_train[cols_to_norm])
X_test[cols_to_norm] = scaler.transform(X_test[cols_to_norm])

n_caracteristicas = X_train.shape[1]

Construcción de los tensores 2D

In [ ]:
X_train_last_59 = X_train.iloc[-59:].copy()

In [ ]:
X_train_last_59

In [ ]:
X_train_last_59.shape

In [ ]:
X_test = pd.concat([X_train_last_59,X_test])

In [ ]:
X_test

In [ ]:
print(X_test.shape)
print(Y_test.shape)

In [ ]:
X_train,Y_train = cnn_data_sequence_pre_train(X_train,Y_train,seque_len=60)

In [ ]:
Y_test

In [ ]:
X_test,Y_test = cnn_data_sequence_pre_train(X_test,Y_test,seque_len=60,previo_aniadido=True)

### Construcción del modelo

In [ ]:
model = Sequential()
model.add(Conv2D(8,(1,n_caracteristicas),activation='relu',input_shape=(60, n_caracteristicas, 1)))
model.add(Conv2D(8, (3, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 1)))
model.add(Conv2D(8, (3, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 1)))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))


print(model.summary())

In [ ]:
model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

### Entrenamiento

In [ ]:
hist = model.fit(X_train,Y_train,batch_size=128,
            validation_data=(X_test,Y_test),
            epochs=100,
            verbose=1,
            shuffle=False)

In [ ]:
mostrarEvolucion(hist)